<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd 
import psycopg2

from sqlalchemy import create_engine, text

import urllib.parse as req
import requests

pd.set_option('display.max_colwidth', None)

In [3]:
connection = f"postgresql+psycopg2://{d['user']}:{d['password']}@{d['host']}:{d['port']}/{d['dbname']}"

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query_vac = f'''select count(*)      -- конкретный столбец можно не выбирать, результат будет одиновым для всех столбцов
            from public.vacancies
'''

In [5]:
df_vac= pd.read_sql_query(query_vac, connection)
df_vac

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
query_emp = f'''select count(*)
            from public.employers
'''

In [7]:
df_emp= pd.read_sql_query(query_emp, connection)
df_emp

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
query_ar = f'''select count(distinct (id))   -- покольку нужны регионы без дублирования счетчик дополняется distinct (id)
            from public.areas
            '''

In [9]:
df_ar= pd.read_sql_query(query_ar, connection)
df_ar

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
query_ind = f'''select count(*)   
            from public.industries
            '''

In [11]:
df_ind= pd.read_sql_query(query_ind, connection)
df_ind

,count
0,294


***

# выводы по предварительному анализу данных

Всего в базе 49 197 вакансии от 23 501 работодателя в 294 сферах.  
Вакансии и работодатели представлены в 1 362 регионах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
query_vac_gr = f'''select 
                    a.name as area,                              -- название региона
                    count(*) as cnt                              -- общий счетчик
                from public.vacancies as v                       -- присоединяем к вакансиям методом join
                    join public.areas as a on v.area_id = a.id   -- регионы, т.к. регионы без вакансии нас не интересуют 
                group by 
                    a.id,
                    a.name
                order by 2 desc
                limit 5                                          -- отлимитируем, чтобы не было большой таблицы
            '''

In [13]:
df_vac_gr = pd.read_sql_query(query_vac_gr, connection)
df_vac_gr

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
query_vac_wage = f'''select 
                           count(v.id) as cnt
                     from public.vacancies as v
                     where
                         v.salary_from is not null or v.salary_to is not null -- сортировка по непустым полям ЗП
            '''

In [15]:
df_vac_wage = pd.read_sql_query(query_vac_wage, connection)
df_vac_wage

,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
query_vac_wage_avg = f'''select 
                           round(avg(salary_from)) as avg_wage                     --  задаем среднюю нижнюю границу, округлив до целого
                         from public.vacancies as v
                         where
                             v.salary_from is not null or v.salary_to is not null
                        union 
                        select 
                           round(avg(salary_to)) as avg_wage                       --  задаем среднюю верхнюю границу, округлив до целого
                         from public.vacancies as v
                         where
                             v.salary_from is not null or v.salary_to is not null
            '''

In [17]:
df_vac_wage_avg = pd.read_sql_query(query_vac_wage_avg, connection)
df_vac_wage_avg

,avg_wage
0,71065.0
1,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
query_vac_sch = f'''select 
                        v.schedule,
                        v.employment,
                        count(v.id) as count_vacancies
                 from public.vacancies as v
                 group by 
                     v.schedule,
                     v.employment
                 order by 3 desc
                ''' 

In [19]:
df_vac_sch = pd.read_sql_query(query_vac_sch, connection)
df_vac_sch

,schedule,employment,count_vacancies
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
query_vac_exp = f'''select 
                        v.experience
                 from public.vacancies as v
                 where v.experience is not null
                 group by 
                     v.experience
                 order by count(v.id)
                ''' 

In [21]:
df_vac_exp = pd.read_sql_query(query_vac_exp, connection)
df_vac_exp

,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

#### выводы по детальному анализу вакансий
 
Пятеркой лидеров по количеству вакансий являются Москва, Санкт-Петербург, Минск, Новосибирск, Алматы.  
Примерно половина вакансий содержит информацию о заработной плате. В среднем заработная плата варьируется в интервале от 71,1 тыс. руб. до 110,5 тыс. рублей.  
В основной массе вакансий требуются соискатели с полной занятостью, меньше всего запросов на проектную работу.  
Чаще всего ищут соискателей с опытом работы от 1 года до 3 лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
query_emp_vac = f'''select
                    e.name
                from 
                    public.employers as e
                    join public.vacancies as v on e.id = v.employer_id
                group by e.name
                order by count(v.id) desc
                limit 5
                
            '''

In [23]:
df_emp_vac = pd.read_sql_query(query_emp_vac, connection)
df_emp_vac

,name
0,Яндекс
1,Ростелеком
2,Тинькофф
3,СБЕР
4,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
query_emp_ar = f'''select
                    a.name as areas,
                    count(distinct (e.id)) as count_employers,
                    count(v.id) as count_vacancies
                from 
                    public.areas as a
                    left join public.vacancies as v on a.id = v.area_id 
                    left join public.employers as e on a.id = e.area
                where v.id is null                                    -- фильтруем по отсутствию вакансий 
                group by a.name
                order by 2 desc
                                
            '''

In [25]:
df_emp_ar = pd.read_sql_query(query_emp_ar, connection)
df_emp_ar.iloc[0][0]

'Россия'

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
query_emp_public = f'''select
                            e.name as employers,
                            count(distinct(v.area_id)) as count_areas              -- добавляем условие уникальности города
                        from 
                            public.employers as e
                            left join public.vacancies as v on e.id = v.employer_id 
                        where v.id is not null                                      -- фильтруем по отсутствию вакансий
                        group by e.name
                        order by 2 desc        
                    '''

In [27]:
df_emp_public = pd.read_sql_query(query_emp_public, connection)
df_emp_public.head()

,employers,count_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
query_emp_ind = f'''select
                            count(distinct(e.id)) employers_not_industry                        -- задаем параметр уникальности работодателя
                        from 
                            public.employers as e
                            left join public.employers_industries as i on e.id = i.employer_id 
                            left join public.industries as ind on ind.id = i.industry_id
                        where ind.name is null                                                  -- фильтруем по отсутствию отрасли
                              
                    '''

In [29]:
df_emp_ind = pd.read_sql_query(query_emp_ind, connection)
df_emp_ind

,employers_not_industry
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
query_emp_ind4 = f'''select
                            distinct e.name
                        from 
                            public.employers as e
                            left join public.employers_industries as i on e.id = i.employer_id 
                            left join public.industries as ind on ind.id = i.industry_id
                        group by e.name
                        having count(ind.id) = 4
                        order by 1 
                        limit 3
                 '''

In [31]:
df_emp_ind4 = pd.read_sql_query(query_emp_ind4, connection)
df_emp_ind4.iloc[-1]

name    2ГИС
Name: 2, dtype: object

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
query_emp_PO = f'''     select
                            count(distinct e.id) PO_employers
                        from 
                            public.employers as e
                            left join public.employers_industries as i on e.id = i.employer_id 
                            left join public.industries as ind on ind.id = i.industry_id
                        where ind.name = 'Разработка программного обеспечения'
                '''

In [33]:
df_emp_PO = pd.read_sql_query(query_emp_PO, connection)
df_emp_PO

,po_employers
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')                        # получаем ссылку
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment) # преобразуем ссылку для пандас

table = pd.read_html(url)                # получаем таблицы со страницы
city_list = tuple(table[1]['Город'])     # выбираем только список городов  и преобразуем в кортеж для возможности вставки в запрос
city_list 

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [35]:
query_emp_mil = f'''     select
                            a.name as areas,
                            count(v.id) as count_vacancies
                        from 
                            public.vacancies as v 
                            left join public.employers as e on e.id = v.employer_id
                            left join public.areas as a on a.id = v.area_id
                        where 
                            e.name = 'Яндекс'
                            and a.name in {city_list}
                        group by a.name
                        union all
                        select 'Total',
                                count(v.id)
                        from
                            public.vacancies as v 
                            left join public.employers as e on e.id = v.employer_id
                            left join public.areas as a on a.id = v.area_id
                        where 
                            e.name = 'Яндекс'
                            and a.name in {city_list}
                        order by 2
                 '''

In [36]:
df_emp_mil = pd.read_sql_query(query_emp_mil, connection)
df_emp_mil

,areas,count_vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

#### выводы по анализу работодателей

Всего в базе 23 501 работодатель.   
В пятерку работодателей по количеству вакансий вхоят Яндекс, Ростелеком, Тинькофф, СБЕР, Газпром нефть.  
Яндекс так же лидирует по количеству регионов, в которых он размещает свои вакансии. В каждом городе-миллионнике представлено от 21 до 54 вакансий. Лидируют в списке по количеству вакансий Москва и Санкт-Петербург. 

Более 100 регионов размещения вакансий так же у Ростелеком и Спецремонт.    
Почти у четверти работодателей не указана сфера деятельности. При этом у 10% работодателей четыре и более сферы деятельности. Разработкой программного обеспечения занимаются 15% работодателей.  


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
query_vac_data = f'''select 
                        count(v.name) as data_count
                     from 
                         public.vacancies as v
                     where 
                         lower(v.name) like '%%data%%' 
                         or lower(v.name) like '%%данн%%'
                    '''

In [38]:
df_emp_data = pd.read_sql_query(query_vac_data, connection)
df_emp_data

,data_count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [40]:
query_vac_data_jun = f'''select 
                            count(*) as junior_data
                         from 
                             public.vacancies as v
                         where 
                             (v.name ilike '%%data scientist%%' 
                             or v.name ilike  '%%data science%%'
                             or v.name ilike  '%%исследователь данных%%'
                             or v.name like  '%%ML%%'                    -- учитываем регистр, что бы в выборку не попали профессии TeamLead
                             or v.name ilike  '%%machine learning%%'
                             or v.name ilike  '%%машинн%обучен%%')
                             and v.name not ilike '%%HTML%%'
                             and
                             (v.name ilike '%%junior%%'
                             or v.experience ilike '%%Нет опыта%%'
                             or v.employment ilike '%%Стажировка%%')
                        '''

In [41]:
df_emp_data_jun = pd.read_sql_query(query_vac_data_jun, connection)
df_emp_data_jun

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\2258418391.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_emp_data_jun = pd.read_sql_query(query_vac_data_jun, connection)


,junior_data
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [42]:
query_vac_data_sql = f'''select 
                            count(*) as data_sql
                         from 
                             public.vacancies as v
                         where 
                             (v.name ilike '%%data scientist%%' 
                             or v.name ilike  '%%data science%%'
                             or v.name ilike  '%%исследователь данных%%'
                             or v.name like    '%%ML%%' 
                             or v.name ilike  '%%machine learning%%'
                             or v.name ilike  '%%машинн%обучен%%')
                             and v.name not ilike '%%HTML%%'
                             and (v.key_skills ilike any(array['%%SQL%%','%%postgres%%']))  -- различные варианты фильтрации
                             '''

In [43]:
df_emp_data_sql = pd.read_sql_query(query_vac_data_sql, connection)
df_emp_data_sql

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\2332656706.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_emp_data_sql = pd.read_sql_query(query_vac_data_sql, connection)


,data_sql
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [44]:
query_vac_data_pt = f'''select 
                            count(*) as data_pt
                         from 
                             public.vacancies as v
                         where 
                             (v.name ilike '%data scientist%' 
                             or v.name ilike  '%data science%'
                             or v.name ilike  '%исследователь данных%'
                             or v.name like    '%ML%' 
                             or v.name ilike  '%machine learning%'
                             or v.name ilike  '%машинн%обучен%')
                             and v.name not ilike '%HTML%'
                             and v.key_skills ilike '%Python%'
                             '''

In [45]:
df_emp_data_pt = pd.read_sql_query(query_vac_data_pt, connection)
df_emp_data_pt

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\934546689.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_emp_data_pt = pd.read_sql_query(query_vac_data_pt, connection)


,data_pt
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [46]:
query_vac_data_skills = f''' select
                                round(avg(length(v.key_skills) - length(replace(v.key_skills, '\t', '')) + 1), 2) as avg_key_skills
                            from 
                                public.vacancies as v
                            where 
                                v.name ilike any(array['%%data scientist%%', '%%data science%%', '%%исследователь данных%%', '%%machine learning%%', '%%машинн%обучен%%'])            
                                or v.name like '%%ML%%' 
                                and v.name not ilike '%%HTML%%'
                            having round(avg(length(v.key_skills) - length(replace(v.key_skills, '\t', '')) + 1), 2) is not null
                          '''

In [47]:
df_emp_data_skills = pd.read_sql_query(query_vac_data_skills, connection)
df_emp_data_skills

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\4017003779.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_emp_data_skills = pd.read_sql_query(query_vac_data_skills, connection)


,avg_key_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [48]:
query_vac_data_wage = f''' select
                               v.experience as experience,
                               round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) as avg_wage
                           from 
                                public.vacancies as v
                            where 
                                (v.name ilike any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])                     
                                or v.name like '%ML%' 
                                and v.name not ilike '%HTML%')
                            group by v.experience
                            order by 2
                       '''

In [49]:
df_emp_data_wage = pd.read_sql_query(query_vac_data_wage, connection)
df_emp_data_wage

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\2547540356.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_emp_data_wage = pd.read_sql_query(query_vac_data_wage, connection)


,experience,avg_wage
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0
3,Более 6 лет,NaN


***

#### выводы по предметному анализу

В базе 1771 вакансия, связанная с данными, из них только 51 (около 4%) для начинающих специалистов.  
Владение SQL или postgres в качестве ключевого навыка содержит 201 (около 11%) вакансия, знание  Python 351 (около 20%) вакансия.
Вакансии для DS в среднем содержат 6-7 ключевых навыков.  
Уровень средней заработной платы колеблется от 74,6 тыс. руб. до 243,1 тыс. руб. в зависимости от опыта работы. Для специалистов со стажем работы более 6 лет уровень заработной платы определяется по итогам собеседования.  

# Общий вывод по проекту

Выведем топ 5 отраслей по количеству вакансий. Дополним сведения количеством работодателей в каждой сфере.

In [71]:
query_ind_vac = f'''select
                        distinct ind.name as industry,
                        count(distinct v.id) as total_vacancies,
                        count(distinct e.id) as total_employers
                    from public.employers as e
                            left join public.employers_industries as i on e.id = i.employer_id 
                            left join public.industries as ind on ind.id = i.industry_id
                            left join public.vacancies as v on v.employer_id = e.id
                    where ind.name is not null
                    group by ind.name
                    order by 2 desc
                    limit 5
                    '''

In [72]:
df_ind_vac = pd.read_sql_query(query_ind_vac , connection)
df_ind_vac

,industry,total_vacancies,total_employers
0,Разработка программного обеспечения,12499,3553
1,"Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг",11034,2993
2,"Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее)",6413,1675
3,Банк,2742,249
4,Мобильная связь,1585,156


Посмотрим какие вакансии содержит сфера 'Разработка программного обеспечения' и потребность рынка в них.

In [73]:
query_ind_vac2 = f'''select
                        e.name as employer,
                        v.name as vacancy,
                        count(v.name) as cout_vacancies
                    from public.industries as i
                        join public.employers_industries as ei on i.id = ei.industry_id
                        left join public.employers as e on ei.employer_id = e.id
                        left join public.vacancies as v on v.employer_id = e.id
                    where i.name = 'Разработка программного обеспечения'
                    group by 
                        i.name, 
                        v.name, 
                        e.name
                    having count(v.name) > 0
                    order by  3 desc
                    '''

In [74]:
df_ind_vac2 = pd.read_sql_query(query_ind_vac2 , connection)
df_ind_vac2.head()

,employer,vacancy,cout_vacancies
0,КРОН,Программист 1С,48
1,Andersen,Ruby Developer,26
2,Ingate,Junior product manager,17
3,ЦИФРОВОЙ ВЫБОР,Ведущий аналитик / консультант 1С по направлению регламентированного учета (БУ/НУ),16
4,Goodt,Дата инженер NiFi (Middle),16


In [55]:
len(list(df_ind_vac2.vacancy.unique()))

6302

Рассмотрим топ регионов по количеству вакансий на удаленной работе.

In [75]:
query_vac_sch2 = f'''select 
                        a.name as cities,
                        count(v.id) as count_vacancies,
                        count(v.employer_id) as count_employers
                 from public.vacancies as v
                      join public.areas as a on v.area_id = a.id
                 where schedule = 'Удаленная работа'
                 group by 
                     a.name
                 order by 2 desc
                 limit 10
                ''' 

In [76]:
df_vac_sch2 = pd.read_sql_query(query_vac_sch2, connection)
df_vac_sch2

,cities,count_vacancies,count_employers
0,Москва,998,998
1,Санкт-Петербург,462,462
2,Нижний Новгород,405,405
3,Новосибирск,345,345
4,Минск,339,339
5,Краснодар,333,333
6,Казань,289,289
7,Екатеринбург,278,278
8,Ростов-на-Дону,272,272
9,Самара,254,254


Рассмотрим топ 5 работодателей с вакансиями на удаленной работе.

In [77]:
query_vac_sch3 = f'''select 
                        e.name as employers,
                        count(v.id) as count_vacancies,
                        count(a.id) as count_areas
                 from public.vacancies as v
                      join public.areas as a on v.area_id = a.id
                      join public.employers as e on v.employer_id = e.id
                 where schedule = 'Удаленная работа'
                 group by 
                     e.name
                 order by 2 desc
                 limit 5
                ''' 

In [78]:
df_vac_sch3 = pd.read_sql_query(query_vac_sch3, connection)
df_vac_sch3

,employers,count_vacancies,count_areas
0,Яндекс,1690,1690
1,DataArt,165,165
2,Тинькофф,95,95
3,Nitka Technologies,77,77
4,ООО ЕФИН,71,71


Уточняем некоторые запросы 

In [60]:
query_vac_data_jun = f'''select 
                            count(*) as junior_data
                         from 
                             public.vacancies as v
                         where 
                             (v.name ilike '%data scientist%' 
                             or v.name ilike  '%data science%'
                             or v.name ilike  '%исследователь данных%'
                             or v.name like  'ML%'                     -- нельзя использовать '%ML%' т.к. попадает QML
                             or v.name ilike  '%machine learning%'
                             or v.name ilike  '%машинн%обучен%')
                             and v.name not ilike '%HTML%'
                             and
                             (v.name ilike any(array['%junior%', '%ученик%', '%стажер%'] )
                             or v.experience ilike '%Нет опыта%'
                             or v.employment ilike '%Стажировка%')
                        
                        '''

In [61]:
df_emp_data_jun = pd.read_sql_query(query_vac_data_jun, connection)
df_emp_data_jun

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\2258418391.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_emp_data_jun = pd.read_sql_query(query_vac_data_jun, connection)


,junior_data
0,47


Количество вакансий для начинающих DS уменьшилось с 51 до 47.

In [62]:
query_vac_data_sql = f'''select 
                             count(*) as data_sql
                         from 
                             public.vacancies as v
                         where 
                             (v.name ilike '%data scientist%' 
                             or v.name ilike  '%data science%'
                             or v.name ilike  '%исследователь данных%'
                             or v.name like    'ML%'                       -- нельзя использовать '%ML%' т.к. попадает QML
                             or v.name ilike  '%machine learning%'
                             or v.name ilike  '%машинн%обучен%')
                             and v.name not ilike '%HTML%'
                             and (v.key_skills ilike any(array['%SQL%','%postgres%']))
                        
                             '''

In [63]:
df_emp_data_sql = pd.read_sql_query(query_vac_data_sql, connection)
df_emp_data_sql

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\2332656706.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_emp_data_sql = pd.read_sql_query(query_vac_data_sql, connection)


,data_sql
0,181


Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres уменьшилось с 201 до 181.

In [64]:
query_vac_data_pt = f'''select 
                            count(*) as data_sql
                         from 
                             public.vacancies as v
                         where 
                             (v.name ilike '%data scientist%' 
                             or v.name ilike  '%data science%'
                             or v.name ilike  '%исследователь данных%'
                             or v.name like    'ML%' 
                             or v.name ilike  '%machine learning%'
                             or v.name ilike  '%машинн%обучен%')
                             and v.name not ilike '%HTML%'
                             and v.key_skills ilike '%Python%'
                             '''

In [65]:
df_emp_data_pt = pd.read_sql_query(query_vac_data_pt, connection)
df_emp_data_pt

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\934546689.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_emp_data_pt = pd.read_sql_query(query_vac_data_pt, connection)


,data_sql
0,320


Количество вакансий, в которых в качестве ключевого навыка указан Python уменьшилось с 351 до 320.

In [66]:
query_vac_data_wage = f''' select
                                                v.experience as experience,
                                                round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) as wage
                                            from 
                                                public.vacancies as v
                                            where 
                                                (v.name ilike any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])                     
                                                or v.name like 'ML%' 
                                                and v.name not ilike '%HTML%')
                                                --and (v.salary_from is not null and v.salary_to is not null)
                                            group by v.experience
                                            order by 2
                                
                          '''

In [67]:
df_emp_data_wage = pd.read_sql_query(query_vac_data_wage, connection)
df_emp_data_wage

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\2547540356.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_emp_data_wage = pd.read_sql_query(query_vac_data_wage, connection)


,experience,wage
0,Нет опыта,74643.0
1,От 1 года до 3 лет,142836.0
2,От 3 до 6 лет,253879.0
3,Более 6 лет,NaN


Средняя заработная плата для соискателей с опытом работы от 1 года до 6 лет незначительно увеличилась.

Подвывод после уточнения запросов: уточнение фильтра вакансий сузило количество вакансий в пределах 10% и скорректировало данные по средней заработной плате.

Посмотрим для скольки вакансий ДС указана зарплатная вилка. 

In [82]:
query_wage_not_null = f'''                  select
                                                --v.experience as experience,
                                                count(coalesce(v.salary_from)) as wage_from_not_null,
                                                count(coalesce(v.salary_to)) as wage_to_not_null,
                                                count(v.name)
                                            from 
                                                public.vacancies as v
                                            where 
                                                (v.name ilike any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])                     
                                                or v.name like 'ML%' 
                                                and v.name not ilike '%HTML%')
                                               and (v.salary_from is not null or v.salary_to is not null)
                                            union
                                            select
                                                --v.experience as experience,
                                                count(coalesce(v.salary_from)) as wage_from_not_null,
                                                count(coalesce(v.salary_to)) as wage_to_not_null,
                                                count(v.name)
                                            from 
                                                public.vacancies as v
                                            where 
                                                (v.name ilike any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])                     
                                                or v.name like 'ML%' 
                                                and v.name not ilike '%HTML%')
                                               and (v.salary_from is not null and v.salary_to is not null)
                             '''

In [83]:
df_wage_not_null = pd.read_sql_query(query_wage_not_null, connection)
df_wage_not_null

C:\Users\зщш\AppData\Local\Temp\ipykernel_9884\135758908.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_wage_not_null = pd.read_sql_query(query_wage_not_null, connection)


,wage_from_not_null,wage_to_not_null,count
0,26,26,26
1,42,41,57


Всего для 57 вакансий указана информация о заработной плате. Из них у 42 вакансий указана верхняя граница, у 41 нижняя. Полная зарплатная вилка указана только у 26 вакансий.

#### Общий вывод по проекту

Всего в базе 49 197 вакансии от 23 501 работодателя в 294 сферах.  
Вакансии и работодатели представлены в 1 362 регионах.  
Часть регионов укрупнены до размера области (например Московская область) или страны (например Россия, Казахстан).   Укрупненные регионы используются для обозначения принадлежности работодателя этому региону, но вакансии размещаются для конкретных городов, даже в случае удаленной работы.  
*Таким образом для будущей модели важно выбирать конкретный город, в котором соискатель хочет найти работу.*

Пятеркой лидеров - регионов по количеству вакансий являются Москва, Санкт-Петербург, Минск, Новосибирск, Алматы.
Пятерку наиболее востребованных сфер деятельности составляют разработка ПО, ИТ-консалтинг, интернет-компании, банки и мобильная связь. Не смотря на технологический характер деятельности, требуются туда специалисты  различных специальностей, не только it-сферы (юристы, бухгалтера и даже 3d артисты).
Всего в сфере разработки программного обеспечения 6 302 уникальные вакансии и 12 499 вакансий в для соискателей.
Примерно половина вакансий содержит информацию о заработной плате. В среднем заработная плата без учета сферы деятельности варьируется в интервале от 71,1 тыс. руб. до 110,5 тыс. рублей.
В основной массе вакансий требуются соискатели с полной занятостью, меньше всего запросов на проектную работу.
В топ 10 городов по количеству вакансий на удаленной работе ожидаемо входят Москва и Санкт-Петербург, кроме того в десятку лидеров вошел Минск. Но у каждого работодателя только 1 вакансия на удаленную работу в каждом городе, где размещена вакансия.
Чаще всего ищут соискателей с опытом работы от 1 года до 3 лет.  
*Таким образом больше всего шансов найти работу в офисе в сфере разработки программного обеспечения жителю крупного города (в идеале миллионника) с опытом работы от 1 года до 3 лет с заработной платой до 110,5 тыс. руб.  
Для будущей модели важно, что технологический характер отрасли работодателя не гарантирует наличие у этого работодателя именно специальности ДС.*  

В пятерку работодателей, лидирующих по количеству вакансий вхоят Яндекс, Ростелеком, Тинькофф, СБЕР, Газпром нефть.
Яндекс так же лидирует по количеству регионов, в которых он размещает свои вакансии. В каждом городе-миллионнике представлено от 21 до 54 вакансий. Лидируют в списке по количеству вакансий Москва и Санкт-Петербург.
Более 100 регионов размещения вакансий так же у Ростелеком и Спецремонт.
Почти у четверти работодателей не указана сфера деятельности. При этом 10% работодателей имеют 4 и более сферы деятельности. Разработкой программного обеспечения занимаются 15% работодателей.  
*Большая часть крупных работодателей (т.е. работодателей с большим количеством вакансий) относится к технологичным отраслям с поиском соискателей в городах-миллионниках. Большое количество работодателей не относят себя к какой-либо сфере деятельности из предложенных (либо игнорируют данную информацию).  
Для модели важно, что при задании конкретной отрасли, из поиска выпадет значительная часть вакансий.*  

В базе 1771 вакансия (т.е. всего 3,5% от общего количества вакансий базы), связанная с данными, из них только 47 (около 4%) для начинающих специалистов.
Владение SQL или postgres в качестве ключевого навыка содержит 181 (около 11%) вакансия, знание Python 320 (около 18%) вакансия. Вакансии для ДС в среднем содержат 6-7 ключевых навыков.
Уровень средней заработной платы колеблется от 74,6 тыс. руб. до 253,9 тыс. руб. в зависимости от опыта работы, что выше среднего уровня заработной платы по всем областям. Для специалистов со стажем работы более 6 лет уровень заработной платы не указан. В целом для профессии характерна вариативность заработной платы и четкая зарплатная вилка указана только в 3% вакансий.  
*Для того, что бы найти работу начинающему ДС специалисту необходимо знание Python, а так же владеть SQL или postgres. Средняя заработная плата начинающего специалиста составляет 74,6 тыс. руб., что выше верхней планки средней заработной платы по рынку в целом. Но скорее всего уровень заработной платы будет определен по итогам собеседования и может быть ниже ожиданий в связи с высокой конкуренцией.  
Для модели важно, что при задании определенного уровня заработной платы может выпасть большая часть вакансий, где оплата труда устанавливается по итогам собеседования.*
